In [1]:
"""
goal: create clean dataset for 2018 - 2021 crashes -- "events" and "drivers"
steps:
1. import dataset
2. remove crashes that occured before 2018
3. remove crashes missing latitude and/or longitude (will be qa/qc'd)
4. remove crashes with incorrect latitude and/or longitude
5. export clean dataset as a geojson file and csv
"""

/Users/shelbygreen/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (0,20,23,29,30,57,59,61,63,64) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


# Events Data

## Import Data

In [1]:
# import libraries
import pandas as pd # for data manipulation
import gspread # to connect with google sheets
from df2gspread import df2gspread as d2g # to interact with google sheets
import geopandas as gpd # for geospatial data manipulation
import os # for file navigation
from datetime import date # for age calculations

In [2]:
# create dataframe of crashes in leon county
df = pd.read_csv("../data/semiclean/leon-events.csv")[['report_number', 'crash_year', 'crash_date_time',
                                       'first_harmful_event', 'total_number_of_vehicles', 'total_number_of_persons',
                                       'latitude', 'longitude', 'on_street_road_highway', 'street_address_number',
                                       'from_intersection_of']]

/Users/shelbygreen/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (29,56,57,58,59,63) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
# read YAML file
import yaml
yaml_file = open("../keys.yaml")
parsed_yaml_file = yaml.load(yaml_file)

<ipython-input-3-eabd164243c5>:4: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  parsed_yaml_file = yaml.load(yaml_file)


## Filter Data

In [6]:
# select crashes that occured on 1/1/2010 or later
df2010 = df.query(" crash_year >= 2010").reset_index(drop=True)

In [7]:
# remove the rows with missing lat/lon data -- these will be addressed in qa/qc
df2010missing = df2010[df2010['latitude'].isna() | df2010['longitude'].isna()]

In [8]:
# export to Google Sheets for QA/QC
from oauth2client.service_account import ServiceAccountCredentials

# access and store credentials
scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']
credentials = ServiceAccountCredentials.from_json_keyfile_name('./gsCredentials.json')
gc = gspread.authorize(credentials)

# connect to the Google Sheet
spreadsheet_key = parsed_yaml_file["spreadsheet_key"]
wks_name = 'missing-2018'

# send data to the Google Sheet
d2g.upload(df2018missing, spreadsheet_key, wks_name, credentials=credentials, row_names=True)

<Worksheet 'missing-2018' id:557274010>

## Clean Data

In [8]:
# add column for crash date
df2010['crash_date'] = df2010['crash_date_time'].str.split(" ").str[0]

# add column for crash time
df2010['crash_time'] = df2010['crash_date_time'].str.split(" ").str[1]

In [9]:
# drop crash date time column
df2010 = df2010.drop(['crash_date_time'], axis=1)

In [10]:
# create clean dataset
df2010clean = df2010[~df2010['latitude'].isna() & ~df2010['longitude'].isna()]
#                      & ~df2018['report_number'].isin(['87845295', '24211781', '24339312'])]

## Export Data

In [9]:
# convert to geodataframe
gdf2018clean = gpd.GeoDataFrame(df2018clean, geometry=gpd.points_from_xy(df2018clean.longitude, df2018clean.latitude))

In [11]:
# export to geojson
gdf2018clean.to_file("./data/clean/all-2018to2021.geojson", driver='GeoJSON')

In [15]:
# export to csv
df2018clean.to_csv("../data/clean/all-2018to2021.csv")

In [11]:
df2010clean

,report_number,crash_year,first_harmful_event,total_number_of_vehicles,total_number_of_persons,latitude,longitude,on_street_road_highway,street_address_number,from_intersection_of,crash_date,crash_time
1,71897779,2010,14.0,2,2,30.046356,-84.028192,N Monroe St,NaN,NaN,2010-12-10,12:00
16,71897893,2010,33.0,1,1,30.444660,-84.289330,US Hwy 90 (W Tennessee St),NaN,NaN,2010-12-10,12:34
51,71898116,2010,14.0,2,2,30.463600,-84.292090,W Tharpe St,NaN,NaN,2010-12-24,12:36
55,71898125,2010,14.0,2,2,30.300099,-84.145080,Raymond Diehl Rd,NaN,NaN,2010-12-25,10:15
66,71898161,2010,10.0,1,2,30.428430,-84.281950,S Adams St,NaN,Jakes and Patterson St,2010-12-29,15:12
...,...,...,...,...,...,...,...,...,...,...,...,...
110577,90116307,2021,14.0,2,3,30.444673,-84.285620,N M L KING JR BLVD,NaN,W TENNESSEE ST,2021-06-29,18:45
110578,90116314,2021,14.0,2,2,30.480562,-84.236873,Capital Circle NE,NaN,WEDNESDAY RD,2021-06-30,12:25
110579,90116316,2021,14.0,2,2,30.431120,-84.226543,Capital Circle SE,NaN,DICK WILSON BLVD,2021-06-30,11:50
110580,90116323,2021,14.0,2,2,30.465070,-84.327236,W Tharpe St,NaN,IVAN DR,2021-06-30,16:40


# Misc

# How many car crashes have happened on Miccosukee 2018-2021?

In [49]:
# crashes within a bounding box
# bounding box from bboxfinder.org 

# miccosukee = df2018clean.query(' on_street_road_highway.str.contains("Miccosukee").values ')
miccosukee[miccosukee['longitude'].between(-84.277496, -84.261682) & miccosukee['latitude'].between(30.445584, 30.456165)].sort_values(["from_intersection_of", "crash_time"], ascending = [True,True])

,report_number,crash_year,first_harmful_event,total_number_of_vehicles,total_number_of_persons,latitude,longitude,on_street_road_highway,street_address_number,from_intersection_of,crash_date,crash_time
22684,89145393,2019,14.0,2,2,30.447141,-84.275703,Miccosukee Rd,NaN,CHERRY ST,2019-10-30,07:27
33044,90111729,2020,14.0,2,4,30.447528,-84.275259,Miccosukee Rd,NaN,CHERRY ST,2020-10-26,10:26
26576,89147106,2020,14.0,2,3,30.447547,-84.275260,Miccosukee Rd,NaN,CHERRY ST,2020-01-07,11:23
14621,87844035,2019,14.0,2,2,30.447410,-84.275384,Miccosukee Rd,NaN,CHERRY ST,2019-01-29,14:08
37704,90115469,2021,14.0,2,2,30.447698,-84.275053,Miccosukee Rd,NaN,CHERRY ST,2021-05-13,20:03
9133,87839457,2018,15.0,3,1,30.454002,-84.265941,Miccosukee Rd,NaN,CRESTVIEW AVE,2018-08-27,13:35
15447,87844863,2019,14.0,2,1,30.454308,-84.265222,Miccosukee Rd,NaN,CRESTVIEW AVE,2019-03-01,17:13
15665,87845116,2019,14.0,2,3,30.453973,-84.265930,Miccosukee Rd,NaN,CRESTVIEW AVE,2019-03-10,17:14
21804,89144515,2019,14.0,2,2,30.454965,-84.262323,Miccosukee Rd,NaN,DOCTORS DR,2019-10-01,11:25
17409,89140096,2019,14.0,3,3,30.449153,-84.273604,Miccosukee Rd,NaN,E BREVARD ST,2019-04-09,17:55


# How many car crashes happened on Black Friday?

In [58]:
len(df2018.query(" crash_year == 2018 and crash_date == '2018-11-30' ").sort_values(by="crash_time", ascending=True))

58

In [57]:
df2018.query(" crash_year == 2019 and crash_date == '2019-11-29' ").sort_values(by="crash_time", ascending=True)

,report_number,crash_year,first_harmful_event,total_number_of_vehicles,total_number_of_persons,latitude,longitude,on_street_road_highway,street_address_number,from_intersection_of,crash_date,crash_time
16659,88151871,2019,18.0,2,3,30.480331,-84.150612,I-10 E,NaN,MM 209,2019-11-29,02:48
23518,89146218,2019,25.0,1,3,30.415489,-84.301143,MILL ST,NaN,SPRINGHILL RD,2019-11-29,04:50
23517,89146218,2019,25.0,1,3,30.415489,-84.301143,MILL ST,NaN,SPRINGHILL RD,2019-11-29,04:50
23516,89146217,2019,10.0,1,2,30.453004,-84.337730,Appleyard Dr,NaN,W TENNESSEE ST,2019-11-29,07:07
25389,89298484,2019,14.0,3,6,30.427297,-84.226852,Capital Circle SE,NaN,APALACHEE PKWY,2019-11-29,07:45
16766,88261649,2019,28.0,1,2,30.481131,-84.370140,INTERSTATE 10 (STATE ROAD 8),NaN,MISSION ROAD,2019-11-29,07:45
23520,89146220,2019,14.0,2,5,30.459131,-84.358870,W Tennessee St,NaN,NaN,2019-11-29,10:30
23519,89146219,2019,14.0,2,2,30.443968,-84.278338,N Gadsden St,NaN,E CALL ST,2019-11-29,11:12
23521,89146221,2019,14.0,2,3,30.444663,-84.278484,E Tennessee St,NaN,N GADSDEN ST,2019-11-29,11:22
23522,89146221,2019,14.0,2,3,30.444663,-84.278484,E Tennessee St,NaN,N GADSDEN ST,2019-11-29,11:22


In [53]:
df2018.query(" crash_year == 2020 and crash_date == '2020-11-27' ").sort_values(by="crash_time", ascending=True)

,report_number,crash_year,first_harmful_event,total_number_of_vehicles,total_number_of_persons,latitude,longitude,on_street_road_highway,street_address_number,from_intersection_of,crash_date,crash_time
26192,88276836,2020,13.0,1,4,30.495797,-84.174679,INTERSTATE 10 (STATE ROAD 8),NaN,US HIGHWAY 90 (STATE ROAD 10),2020-11-27,00:00
33641,90112339,2020,14.0,2,3,30.456258,-84.276484,N Gadsden St,5,E 6TH AVE,2020-11-27,10:35
33640,90112337,2020,14.0,2,5,30.461008,-84.225636,Mahan Dr,NaN,CAPITAL CIR NE,2020-11-27,11:03
33653,90112351,2020,14.0,2,2,30.542517,-84.228066,Thomasville Rd,NaN,KERRY FOREST PKWY,2020-11-27,11:19
26392,88349799,2020,14.0,2,2,30.489695,-84.164890,INTERSTATE 10 (STATE ROAD 8) MM 209 WB,NaN,US 90 (MAHAN RD),2020-11-27,12:00
25588,83822610,2020,14.0,3,5,30.487501,-84.161654,INTERSTATE 10,NaN,209MM WB ENTRANCE RAMP,2020-11-27,12:33
26091,88260935,2020,14.0,2,3,30.484682,-84.157933,INTERSTATE 10 (STATE ROAD 8),NaN,U.S. HIGHWAY 90 (STATE ROAD 10),2020-11-27,12:45
30659,89769418,2020,37.0,1,0,30.437897,-84.309217,Belle Vue Way,1607,RIVOLI RD,2020-11-27,14:51
33642,90112340,2020,14.0,2,5,30.412602,-84.305676,W Orange Ave,NaN,LAKE MUNSON ST,2020-11-27,15:08
33643,90112341,2020,36.0,1,2,30.443816,-84.249778,N Blair Stone Rd,NaN,BLAIR STONE RD,2020-11-27,15:22


In [56]:
test_df.groupby('crash_year').count().sort_values(ascending=False, by="report_number")
#.query(" report_number == 90115367 ")
# pd.options.display.max_columns = 999

# driver_distraction_code
# driver_condition_code
# address
# city
# zip code
# date of birth
# sex
# helmet_code
# air_bag_deployment_code
# ejected
# suspected_alcohol_use_code
# suspected_drug_use_code
# injurty_severity
# ems_transport_type
# driver_action_1

# how many visitors/out-of-towners get into car accidents?

,report_number,vehicle_number,person_number,recommend_re_exam,driver_license_state,dl_expiration_date,driver_license_type,driver_distraction_code,driver_condition_code,required_endorsements,vision_obstructed,insurance_company,city,state,zip_code,date_of_birth,sex,helmet_code,restraint_system_code,eye_protection_code,air_bag_deployment_code,ejected,suspected_alcohol_use_code,alc_tested_code,alc_test_type_code,alc_test_results,blood_alcohol_content,suspected_drug_use_code,drug_tested_code,drug_test_type_code,drug_test_results,injury_severity,ems_transport_type,ems_name,ems_run_number,injured_taken_location,seat_position_code,row_position_code,other_position_code,driver_action_1,driver_action_2,driver_action_3,driver_action_4
crash_year,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2018,1329812,1329812,1329812,1134582,1285358,1271135,1290593,1320563,1319423,1011907,1296655,1303474,1305706,1302528,1300361,1284898,1309237,337510,1295624,795041,1293698,1315753,1319572,504084,11254,11003,6991,1319245,498962,3894,3724,1302700,1305247,123672,75720,96159,1318754,1317533,714258,1307302,53119,16583,10773


In [15]:
# df2018.groupby('first_harmful_event').count().sort_values(ascending=False, by="report_number")
# df2018.groupby('crash_year').count().sort_values(ascending=False, by="report_number")

# first harmful event
# 14 - motor vehicle in transport (another car)
# 15 - parked motor vehicle
# 39 - "Other Fixed Object (wall, building, tunnel, etc.)"
# 18 - Other Non-Fixed Object
# 32 - Tree (standing)
# 10 - Pedestrian
# 36 - "Other Post, Pole or Support"
# 11 - pedalcycle

# Drivers data
report number -- there could be multiple drivers under one report number

## Import Data

In [2]:
# define path where the driver tables are stored
path = "../data/raw/"

# create empty list
li = []

# for loop -- loop through each file and save the data
for filename in os.listdir(path):
    temp_df = pd.read_csv(os.path.join(path, filename), index_col=None, header=0) \
    .drop(['dl_number', 'policy_number', 'first_name', 'middle_name', 'last_name', 'suffix', 'phone_number', 'address'], axis=1)
    li.append(temp_df)
    
# create dataframe for the drivers table, 2018 - 2021
driver_df = pd.concat(li, axis=0, ignore_index=True)

## Filter Data

In [10]:
# create list of record numbers from the events table
id_list = list(df2018['report_number'])

In [11]:
# remove rows from the list that don't match the ones from the events table
driver_df = driver_df.query(" report_number in @id_list")

In [12]:
driver_df

,report_number,crash_year,vehicle_number,person_number,recommend_re_exam,driver_license_state,dl_expiration_date,driver_license_type,driver_distraction_code,driver_condition_code,...,ems_name,ems_run_number,injured_taken_location,seat_position_code,row_position_code,other_position_code,driver_action_1,driver_action_2,driver_action_3,driver_action_4
9,87503393,2018,1,1,1.0,FL,08-JUN-24,5.0,1.0,1.0,...,NaN,NaN,NaN,1.0,1.0,NaN,4.0,NaN,NaN,NaN
10,87503393,2018,2,2,1.0,FL,08-JUL-25,5.0,1.0,1.0,...,NaN,NaN,NaN,1.0,1.0,NaN,1.0,NaN,NaN,NaN
397,87503396,2018,1,1,1.0,VA,15-JAN-24,5.0,1.0,1.0,...,NaN,NaN,NaN,1.0,1.0,NaN,2.0,NaN,NaN,NaN
398,87503397,2018,1,1,1.0,FL,28-JAN-25,5.0,1.0,1.0,...,NaN,NaN,NaN,1.0,1.0,NaN,10.0,NaN,NaN,NaN
399,87503397,2018,2,2,1.0,FL,23-MAY-20,5.0,1.0,1.0,...,NaN,NaN,NaN,1.0,1.0,NaN,1.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4315689,90115800,2021,1,1,1.0,FL,03-AUG-28,5.0,1.0,1.0,...,NaN,NaN,NaN,1.0,1.0,NaN,3.0,NaN,NaN,NaN
4315690,90115802,2021,1,1,1.0,FL,24-DEC-27,5.0,4.0,1.0,...,NaN,NaN,NaN,1.0,1.0,NaN,2.0,NaN,NaN,NaN
4315691,90115802,2021,2,2,1.0,FL,09-DEC-28,5.0,1.0,1.0,...,NaN,NaN,NaN,1.0,1.0,1.0,1.0,NaN,NaN,NaN
4315692,90115805,2021,1,1,1.0,FL,06-JUL-21,5.0,1.0,1.0,...,NaN,NaN,NaN,1.0,1.0,1.0,3.0,NaN,NaN,NaN


## Clean Data

In [83]:
# change date of birth type to date


In [84]:
# create age of driver column

In [78]:
# create determine age function
def calculateAge(birthDate):
    today = date.today()
    age = today.year - birthDate.year - ((today.month, today.day) < (birthDate.month, birthDate.day))
 
    return age